<a href="https://colab.research.google.com/github/mathjoha/strik-og-kod/blob/main/notebooks/KnC_handson_workbook_swe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
[![Open In Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/mathjoha/strik-og-kod.git/main?labpath=notebooks%2KnC_handson_workbook_swe.ipynb) 

# Knit & Code
author: "Mathias Johansson, Joakim Axelsson & Max Odsbjerg Pedersen"

date: "2025-04-08"

Detta dokument består av den kod-delen av workshoppen "Knit and Code" vid
Humanistiska och Teologiska Fakulteterna vid Lunds universitet utvecklad I
samarbete med _AU Bibliotek vid Det Kongelige Bibliotek_. Workshopen handlar om
att dra paralleller mellan stickning och kodning. "Kodning" förstås här som
kopplingen mellan kodningsbaserad datahantering och ligger därför inom området
datorvetenskap. Eftersom workshopen är gjord i sammanhang av humaniora avser
exemplet _textmining_. När man använder _textmining_ är det primära intresset
att extrahera information ur stora korpus - vilket är exakt det intresse som
många humanister har.

Inget recept är komplett utan en bild av den färdiga produkten som en av de
första punkterna. Och detta är inget undantag. Slutresultatet i slutet av detta
dokument är visualiseringen som visas precis under denna paragraf. Den visar de
vanligast förekommande orden i gamla tidningsartiklar om stickning efter att
alla stoppord har tagits bort (det, att, till, och, i - ord som inte bär någon
större betydelse).

![](https://github.com/mathjoha/strik-og-kod/blob/53a64caa55cebfea9e3d9e2db1f7305d1043d129/notebooks/swe_wc.png?raw=true)

Yttrade ABM-ord och deras sällskap.

<br>



## Ladda ner och installera Python paket
Vi arbetar i programmeringsspråket [Python ](https://www.python.org/), ett
gratis och _open-source_ programmeringsspråk. Python får mest av sin
funktionalitet genom att importera 'bibliotek', och python har ett mycket brett
ekosystem med bibliotek som erbjuder nästan all funktionalitet du kan tänka
dig. Bland annat många möjligheter för att bearbeta text, statistik och grafisk
presentation av resultaten. Python får mest av sin funktionalitet genom att
importera 'bibliotek', och python har ett mycket starkt ekosystem med bibliotek
som erbjuder nästan alla funktioner du kan tänka dig. Bland annat många
möjligheter för att bearbeta text, statistik och grafisk presentation av
resultaten.

I denna workshop är de relevanta paketen:
- Pandas: Ett kraftfullt bibliotek för datahantering.
- Wordcloud: Ett Python-bibliotek för att generera ordmoln från text.
- Matplotlib: Ett bibliotek för att skapa statiska, animerade och interaktiva
  visualiseringar i Python.

Vi kommer att installera dessa paket med hjälp av pip, Pythons pakethanterare.
Pip är ett kommandoradsverktyg som gör det lätt att installera och hantera
Python-paket.

Vi använder också två bibliotek från _standardbiblioteket_:
- re: Ett reguljärt uttrycksmodul -- för textmönstermatchning.
- Counter: Ett objekt för att räkna förekomster av objekt.


## Data – uttalanden about ABM

The första vi behöver är någon text att behandla. Vi kommer här att använda
_uttalanden_ från Svenska riksdagsdebatter från 1867-2022. För att hitta och
komma åt dessa _uttalanden_ kommer vi först använda
[Riksdagsdebatter.se](https://riksdagsdebatter.se/public/index.html#/about) som
erbjuder ett grafiskt gränssnitt för sökning och åtkomst till debatterna. För
att hitta alla uttalanden som nämner ABM söker vi efter följande tre nyckelord:


- `arkiv*`
- `bibliotek*`
- `muse*`

Asterisk `*` markerar trunkering -- expanderar för att hitta *alla ord* som
börjar med `arkiv`, `bibliotek` eller `muse` (1477 ord enligt GUI).

`Riksdagsdebatter.se` erbjuder inte något API och de ~9k resultaten är
paginerade vid 50 resultat per sida. Men källmaterialet finns tillgängligt på:
[the-swedish-parliament-corpus](https://github.com/swerik-project/the-swedish-parliament-corpus)
where one can download the entire corpus and all speaker-metadata. In där man
kan ladda ner hela korpusen och all talarmetadata. I förberedelse för denna
workshop har jag laddat ner posterna, filterat bort alla uttalanden som INTE
nämner minst ett av våra nyckelord och omstrukturerat resultatet till en
CSV-fil.
[skript](https://gist.github.com/mathjoha/edcdaf57c5c2d58d9f6b58a6350b811d)
[korpus](https://raw.githubusercontent.com/mathjoha/strik-og-kod/refs/heads/main/the-swedish-parliament-corpus_ABM_.csv)



### Hämta data

För att komma åt datat för workshoppen behöver det laddas ner, vilket 
lätt kan göras med `wget`-programmet så här:

In [ ]:
!wget -c -nc "https://raw.githubusercontent.com/mathjoha/strik-og-kod/08f31ff60c9460efb232fecc1fbd07aace62dab4/data/the-swedish-parliament/the-swedish-parliament-corpus_ABM_.csv" -O corpus.csv

Sedan använder vi funktionen `read_csv` från pandas-biblioteket för att ladda
innehållet i filen till en DataFrame och behålla det i minnet under
variabelnamnet "strik". En DataFrame är jämförbar med ett kalkylblad eftersom
det är en stor matris som lagrar data vi arbetar med.



Detta ger oss en ny Pandas DataFrame som heter "riksdag" och innehåller 13755
rader och 5 kolumner.

Vad som är särskilt intressant för oss är kolumnen "content" – det är här de
transkriberade yttrandena lagras. En del av denna text kommer inte att vara
lätt att läsa eftersom den innehåller fel, och det är här du möter den första
nackdelen med att arbeta med digitaliserad text: OCR-fel.



För att förstå varför dessa fel uppstår är det nödvändigt att vända sig till
digitaliseringen. I denna process skannas protokollen och bearbetas med en
OCR-motor. Dessa motorer tenderar att fungera i två huvudsakliga steg:

  1. Segmentering av bilden i olika textblock.
  2. Transkribering av textblocken till.

Dessa motorer är vanligtvis utvecklade och testade på nyare material - delvis
för att det äldre materialet är mer komplext, och det finns mindre av det
tillgängligt.

Teamet bakom `The Swedish Parliament Corpus` har lagt ned mycket arbete på att
utveckla system för att transkribera de äldre debatterna, identifiera talare
och mappa dem till metadata. Ändå finns många
[OCR-fel kvar](https://github.com/swerik-project/pyriksdagen/blob/5cdc0875b7ed9a46f6ec7039d439d6e22e6acf54/examples/corpus-walkthrough.ipynb)
i materialet.

## Text mining-uppgiften

Först kommer vi att konvertera textenrna till gemener och dela upp dem i ord
med hjälp av
[Regular expression](https://sv.wikipedia.org/wiki/Reguljärt_uttryck). Vi
lagrar sedan dessa `listor` av gemena ord i DataFrame i kolumnen `ord` och vi
expanderar denna kolumn till en ny dataframe där varje ord har sin egen rad.


Låt oss skriva ut den nya dataframen för att se hur tidytext-formatet ser ut i
praktiken. Detta uppnås genom att skriva namnet på datafamen:

Om vi bläddrar igenom kolumnerna (med den lilla svarta pilen i högra hörnet)
kommer den sista kolumnen nu att vara "ord" som endast innehåller enskilda ord.
## Analys

### Molntext

För att få en överblick av vår datamängd börjar vi med att räkna de vanligaste
orden i protokollen som nämner ABM under perioden 1867 till 2022:


Till ingen överraskning är de vanligaste orden i datamängden grammatiska
partiklar. Ett sätt att negera dessa ord är genom att använda en stopwordlista
som kan användas för att ta bort oönskade ord. För detta kommer vi att använda
en stopwords-lista som publicerats offentligt av
[@peterdalle](https://gist.github.com/peterdalle):

In [ ]:
!wget "https://gist.githubusercontent.com/peterdalle/8865eb918a824a475b7ac5561f2f88e9/raw/cc1d05616e489576c1b934289711f041ff9b2281/swedish-stopwords.txt" -O stopord.txt

Vi filtrerar bort alla stoppord och tittar igen.

Vi kan redan se ganska många intressanta ord. Det är många fler yttranden om
museer än om arkiv, och vi kan inte ens se arkiv i 1000 antal. Det finns också
ett flertal ord som inte är vardagliga ord men är väldigt vanlika i samanhanget
av riksdagsrdebatter och ett fåtal OCR-fel.

Men en lista är lite tråkig att titta på. Kunde vi kanske skapa ett
vackert ordmoln? Naturligtvis kan vi det!